# Spotify Graph Queries

We perform ten queries on our Spotify database relying on ***SPARQLWrapper***.

## Setup

We import all the necessary libraries and we set the URL needed by SPARQLWrapper

In [ ]:
# Imports

import pandas as pd
from pandas import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
SPOTIFY_ONTOLOGY = "https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#"
REPOSITORY_NAME = "eu"

SPARQL_ENDPOINT = "http://localhost:7200/repositories/" + REPOSITORY_NAME

sparql = SPARQLWrapper(SPARQL_ENDPOINT)


## Query utilities
We define a function to perform a SPARQL query

In [ ]:
# Execute a SPARQL query and return a DataFrame
def executeQuery(query, maxRows=None):
    global sparql

    # Set the number of rows and columns to display
    pd.set_option("display.max_rows", maxRows, "display.max_columns", None)

    # Set the query
    sparql.setQuery("""
            prefix so: <https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#>
            prefix countries: <http://eulersharp.sourceforge.net/2003/03swap/countries#>
            prefix xsd: <http://www.w3.org/2001/XMLSchema#>

            {query}
        """.format(query=query))

    # Set the result format
    sparql.setReturnFormat(JSON)

    # Execute the query
    results = sparql.query().convert()

    # If it is an ask
    if "boolean" in results.keys():
        return results["boolean"], "ask"
    
    # If it is a select
    if "results" in results.keys():
        # Create a DataFrame from the returned JSON
        resultDF = json_normalize(results["results"]["bindings"])

        # Remove the datadtypes columns
        resultDF = resultDF[[col for col in resultDF.columns if not col.endswith(
            ".type") and not col.endswith(".datatype")]]

        # Remove .value from column names
        resultDF = resultDF.rename(columns=lambda col: col.replace(".value", ""))

        # Remove the spotify ontology prefix from uris (to get only the ID)
        resultDF = resultDF.applymap(lambda x: str(x).lstrip(SPOTIFY_ONTOLOGY))

        return resultDF, "select"
    
    return None


We define a function to print the result of a query

In [ ]:
def printResult(result, resultType):
    if resultType == "select":
        print("🗃️ [[RESULT]]\n")
        result

    if resultType == "ask":
        print("🎱 [[ANSWER]]\n")
        print("✔️ Yes" if result else "❌ No")


## Queries

#### Query 1: On average how many artist of a specific nationality are in the top 100 of the same country ?


In [ ]:
result, resultType = executeQuery("""
SELECT ?country (xsd:integer(AVG(?num_artists)) AS ?avg_num_artists) WHERE {
	{
		SELECT ?date ?country (COUNT(DISTINCT ?artist) AS ?num_artists) WHERE{
            ?person 		so:isMemberOf ?artist ;
                    		so:hasNationality ?country.
            ?artist 		so:partecipateIn ?track ;
                    		so:name ?name.
            ?track 			so:appearsIn ?appeareance .
            ?appeareance 	so:isPositionedIn ?chart .
            ?chart 			so:isReferredTo ?country ;
                   			so:date ?date .   
    	}
    	GROUP BY ?date ?country
	}
}
GROUP BY ?country
ORDER BY DESC(?avg_num_artists)
""")

printResult(result, resultType)